### README
This Notebook was created by Carlo Occhiena and is intended as an appendix to the degree thesis at the University of Genoa - Faculty of Economics.

Code bibliography and documentation is quoted in the thesis document.


# Brownian Motion
## Chapter II

In [ ]:
"""
Naive implementation of a simple iteration following the Brownian motion
definition
"""

"""
scipy.stats.norm
norm = <scipy.stats._continuous_distns.norm_gen object>[source]

A normal continuous random variable.
The location (loc) keyword specifies the mean.
The scale (scale) keyword specifies the standard deviation.
"""
from scipy.stats import norm

# Process parameters
delta = 0.25
dt = 0.1

# Initial condition.
x = 0.0

# Number of iterations to compute.
n = 5

# Iterate to compute the steps of the Brownian motion.
for k in range(n):
    x = x + norm.rvs(scale=delta**2*dt)
    print(x)

In [ ]:
"""
brownian() implements one dimensional Brownian motion (i.e. the Wiener process).
"""

from math import sqrt
from scipy.stats import norm
import numpy as np


def brownian(x0, n, dt, delta, out=None):
    """
    Generate an instance of Brownian motion (i.e. the Wiener process):

        X(t) = X(0) + N(0, delta**2 * t; 0, t)

    where N(a,b; t0, t1) is a normally distributed random variable with mean a and
    variance b.  The parameters t0 and t1 make explicit the statistical
    independence of N on different time intervals; that is, if [t0, t1) and
    [t2, t3) are disjoint intervals, then N(a, b; t0, t1) and N(a, b; t2, t3)
    are independent.

    Written as an iteration scheme,

        X(t + dt) = X(t) + N(0, delta**2 * dt; t, t+dt)


    If `x0` is an array (or array-like), each value in `x0` is treated as
    an initial condition, and the value returned is a numpy array with one
    more dimension than `x0`.

    Arguments
    ---------
    x0 : float or numpy array
        The initial condition(s) (i.e. position(s)) of the Brownian motion.
    n : int
        The number of steps to take.
    dt : float
        The time step.
    delta : float
        delta determines the "speed" of the Brownian motion.  The random variable
        of the position at time t, X(t), has a normal distribution whose mean is
        the position at time t=0 and whose variance is delta**2*t.
    out : numpy array or None
        If `out` is not None, it specifies the array in which to put the
        result.  If `out` is None, a new numpy array is created and returned.

    Returns
    -------
    A numpy array of floats with shape `x0.shape + (n,)`.

    Note that the initial value `x0` is not included in the returned array.
    """

    x0 = np.asarray(x0)

    # For each element of x0, generate a sample of n numbers from a
    # normal distribution.
    r = norm.rvs(size=x0.shape + (n,), scale=delta*sqrt(dt))

    # If `out` was not given, create an output array.
    if out is None:
        out = np.empty(r.shape)

    # This computes the Brownian motion by forming the cumulative sum of
    # the random samples.
    np.cumsum(r, axis=-1, out=out)

    # Add the initial condition.
    out += np.expand_dims(x0, axis=-1)

    return out

In [ ]:
"""
Plot several realizations of Brownian motion calling iterativly
the function brownian()
"""

import numpy
from pylab import plot, show, grid, xlabel, ylabel, title


delta = 2                   # The Wiener process parameter.
T = 10.0                    # Total time.
N = 500                     # Number of steps.
dt = T/N                    # Time step size
m = 20                      # Number of realizations to generate.
x = numpy.empty((m,N+1))    # Create an empty array to store the realizations.
x[:, 0] = 50                # Initial values of x.

brownian(x[:,0], N, dt, delta, out=x[:,1:])

t = numpy.linspace(0.0, N*dt, N+1)
for k in range(m):
    plot(t, x[k], lw=0.8)
title('Simulated Geometric Brownian Motion Path')
xlabel('t', fontsize=16)
ylabel('x', fontsize=16)
grid(True)
show()

In [ ]:
"""
Brownian motion simulation using
Euler-Maruyama method.

In Itô calculus, the Euler–Maruyama method (also simply called the Euler method)
is a method for the approximate numerical solution of a stochastic differential
equation (SDE).
It is an extension of the Euler method for ordinary differential equations to
stochastic differential equations,
named after Leonhard Euler and Gisiro Maruyama.
However, it should be noted that the same generalisation
cannot be done for any arbitrary deterministic method.

"""

import numpy as np
import matplotlib.pyplot as plt

# Parameters
num_paths = 1000                  # Number of Brownian motion paths
num_steps = 1000                  # Number of time steps
T = 1.0                           # Time horizon
dt = T / num_steps                # Time step size
t = np.linspace(0, T, num_steps)  # Time intervals

# Euler-Maruyama method for Brownian motion
np.random.seed(42)  # For reproducibility
dW = np.sqrt(dt) * np.random.randn(num_paths, num_steps)  # Increments
W = np.cumsum(dW, axis=1)  # Cumulative sum to build W(t)

# Add initial value W(0) = 0
W = np.hstack((np.zeros((num_paths, 1)), W))

# Plot a sample of Brownian motion paths
plt.figure(figsize=(10, 6))
for i in range(min(30, num_paths)):  # Show only 30 trajectories
    plt.plot(t, W[i, :-1])
plt.title("Brownian Motion Simulations (Euler Method) [1'000 paths]")
plt.xlabel("Time")
plt.ylabel("$W(t)$")
plt.grid(True)
plt.show()

# Normal distribution of Brownian motion at t = T
final_values = W[:, -1]
plt.figure(figsize=(8, 6))
plt.hist(final_values, bins=30, density=True, alpha=0.7, label="Simulated distribution")
mean, std_dev = 0, np.sqrt(T)
x = np.linspace(-4 * std_dev, 4 * std_dev, 1000)
plt.plot(x, (1 / (np.sqrt(2 * np.pi) * std_dev)) * np.exp(-x**2 / (2 * std_dev**2)),
         label="Theoretical normal distribution")
plt.title("Distribution of Final Values of Brownian Motion ($t = T$)")
plt.xlabel("$W(T)$")
plt.ylabel("Probability Density")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
"""
Ideal graphical representation of the physical process of Brownian motion.
The motion of a particle in a liquid, inside a solid container, is simulated.

"""

import numpy as np
import matplotlib.pyplot as plt

# Parameters
R = 5.0          # Radius of the cup
H = 10.0         # Height of the cup
N_steps = 10000  # Number of steps in the simulation
delta = 0.1      # Standard deviation of the step size

# Initialize position
x = [0.0]
y = [0.0]
z = [H / 2.0]  # Start at half the height of the cup

# Simulate Brownian motion
for _ in range(N_steps):
    # Generate random displacements
    dx, dy, dz = np.random.normal(0, delta, 3)

    # Tentative new position
    x_new = x[-1] + dx
    y_new = y[-1] + dy
    z_new = z[-1] + dz

    # Check radial boundary (cup walls)
    r_new = np.sqrt(x_new**2 + y_new**2)
    if r_new > R:
        # Reflect off the wall
        x_new = x[-1]
        y_new = y[-1]

    # Check vertical boundaries (base and open top)
    if z_new < 0:
        # Reflect off the base
        z_new = -z_new
    elif z_new > H:
        # Reflect off the top edge
        z_new = 2 * H - z_new

    # Append new position
    x.append(x_new)
    y.append(y_new)
    z.append(z_new)

# Convert lists to arrays for plotting
x = np.array(x)
y = np.array(y)
z = np.array(z)

# Plotting the trajectory
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Plot the particle's path
ax.plot(x, y, z, lw=0.5, color='red')

# Set the axes limits
ax.set_xlim([-R, R])
ax.set_ylim([-R, R])
ax.set_zlim([0, H])

# Plot the cup boundary (cylinder surface)
theta = np.linspace(0, 2 * np.pi, 100)
z_cylinder = np.linspace(0, H, 50)
theta_grid, z_grid = np.meshgrid(theta, z_cylinder)
x_cylinder = R * np.cos(theta_grid)
y_cylinder = R * np.sin(theta_grid)
ax.plot_surface(x_cylinder, y_cylinder, z_grid, alpha=0.1, color='gray')

# Labels and title
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')
plt.title('Brownian Motion of a particle in a glass of water')

plt.show()

# Black & Scholes
## Chapter II

In [ ]:
"""
Option pricing applying Black and Scholes
explicit formula.

"""
import math
from scipy.stats import norm

# variable assignment
S = 45       # Underlying Price (Asset)
K = 40       # Strike Price (Option Cost)
T = 2        # Time to Maturity (Option Expiry Date)
r = 0.1      # Risk-Free Rate (T-Bond)
vol = 0.1    # St Dev (σ)

# d1 and d2 probability factors
d1 = (math.log(S/K) + (r + 0.5 * vol**2)*T ) / (vol * math.sqrt(T))
d2 = d1 - (vol * math.sqrt(T))

# theoretical price of a call (C) option
C = S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)

# theoretical price of a put (P) option
P = K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

print('The value of d1 is: ', round(d1, 4))
print('The value of d2 is: ', round(d2, 4))
print('The price of the call option is: $', round(C, 2))
print('The price of the put option is: $', round(P, 2))

In [ ]:
"""
Implied volatility calculation using the Black-Scholes formula.
Volatility surface plotted on real stocks or indexes.
For the purpose of the thesis, observations were made on SP500 and
Microsoft stocks.

"""

import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt

import matplotlib.pyplot as plt

def option_chains(ticker):
    asset = yf.Ticker(ticker)
    expirations = asset.options

    chains = pd.DataFrame()

    for expiration in expirations:
        # tuple of two dataframes
        opt = asset.option_chain(expiration)

        calls = opt.calls
        calls['optionType'] = "call"

        puts = opt.puts
        puts['optionType'] = "put"

        chain = pd.concat([calls, puts])
        chain['expiration'] = pd.to_datetime(expiration) + pd.DateOffset(hours=23, minutes=59, seconds=59)

        chains = pd.concat([chains, chain])

    chains["daysToExpiration"] = (chains.expiration - dt.datetime.today()).dt.days + 1

    return chains

# yfinance provides an estimate of implied volatility. We pick Microsoft (MSFT)
# or SP500 (SPY)
options = option_chains("SPY")

calls = options[options["optionType"] == "call"]

# print the expirations
set(calls.expiration)

# select an expiration to plot
calls_at_expiry = calls[calls["expiration"] == "2027-12-17 23:59:59"]

# filter out low vols
filtered_calls_at_expiry = calls_at_expiry[calls_at_expiry.impliedVolatility >= 0.000001]

# set the strike as the index so pandas plots nicely
filtered_calls_at_expiry[["strike", "impliedVolatility"]].set_index("strike").plot(
    title="Implied Volatility Skew", figsize=(7, 4)
)

# select an expiration to plot
calls_at_strike = options[options["strike"] == 571.0]

# filter out low vols
filtered_calls_at_strike = calls_at_strike[calls_at_strike.impliedVolatility >= 0.00001]

# set the strike as the index so pandas plots nicely
filtered_calls_at_strike[["expiration", "impliedVolatility"]].set_index("expiration").plot(
    title="Implied Volatility Term Structure", figsize=(7, 4)
)

# pivot the dataframe
surface = (
    calls[['daysToExpiration', 'strike', 'impliedVolatility']]
    .pivot_table(values='impliedVolatility', index='strike', columns='daysToExpiration')
    .dropna()
)

# create the figure object
fig = plt.figure(figsize=(10, 8))

# add the subplot with projection argument
ax = fig.add_subplot(111, projection='3d')

# get the 1d values from the pivoted dataframe
x, y, z = surface.columns.values, surface.index.values, surface.values

# return coordinate matrices from coordinate vectors
X, Y = np.meshgrid(x, y)

# set labels
ax.set_xlabel('Days to expiration')
ax.set_ylabel('Strike price')
ax.set_zlabel('Implied volatility')
ax.set_title('SP500 Call implied volatility surface exp 25-01-2025')

# plot
plt.tight_layout()
ax.plot_surface(X, Y, z, cmap='plasma', edgecolor='none', antialiased=True)
plt.show()

###Implied volatility annex:
Checks for data consistency. \
In order to pick the right expiry and strike, we need first to scan market data


In [ ]:
# print strike prices and implied vol for the available options
print(calls[['strike', 'impliedVolatility']].head())

In [ ]:
# print expiry dates of the options
print(f"Exp:{calls.expiration}")

In [ ]:
# print the range of available options
print(f"Options:{options}")

# Dummy data for B&S Implied Vol surface

Since market data expose to a lot of volatility (of course!) in order to obtain a good-looking chart, it is also provided a dummy-data example


In [ ]:
# Imports
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Parameters
T = 1.0      # Time to maturity
K = 65.0     # Strike price
r = 0.05     # Risk-free rate
sigma = 0.5  # Volatility

# Grid settings
S_max = 100  # Maximum stock price
S_min = 0    # Minimum stock price
N = 100      # Number of time steps
M = 100      # Number of stock price steps

# Time and stock price grids
dt = T / N
dS = (S_max - S_min) / M
t_grid = np.linspace(0, T, N + 1)
S_grid = np.linspace(S_min, S_max, M + 1)

# Black-Scholes PDE solution for a European call option
def black_scholes_call(S, t):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * (T - t)) / (sigma * np.sqrt(T - t))
    d2 = d1 - sigma * np.sqrt(T - t)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * (T - t)) * norm.cdf(d2)
    return call_price

# Generate the surface
C = np.zeros((M + 1, N + 1))
for i in range(M + 1):
    for j in range(N + 1):
        C[i, j] = black_scholes_call(S_grid[i], t_grid[j])

# Plotting
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')
S, T = np.meshgrid(S_grid, t_grid)
surf = ax.plot_surface(S, T, C.T, cmap='plasma')

ax.set_xlabel('Stock Price')
ax.set_ylabel('Time to Maturity')
ax.set_zlabel('Option Price')
ax.set_title('Black-Scholes Option Pricing Surface')

fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, label="Option Price")

# Rotate and adjust aspect ratio
ax.view_init(elev=20, azim=210)  # Adjust elevation and azimuthal angles
ax.set_box_aspect([2, 1, 1])     # Adjust aspect ratio


plt.show()

# Heston Model
## Chapter IV

In [ ]:
# A fast, vectorized approach to calculating
# Implied Volatility and Greeks using the Black,
# Black-Scholes and Black-Scholes-Merton pricing

!pip install py_vollib_vectorized

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from py_vollib_vectorized import vectorized_implied_volatility as implied_vol

# Parameters
# simulation dependent
S0 = 100.0             # asset price
T = 1.0                # time in years
r = 0.05               # risk-free rate
N = 252                # number of time steps in simulation
M = 1000               # number of simulations

# Heston dependent parameters
kappa = 2.7            # rate of mean reversion of variance under risk-neutral dynamics
theta = 0.20**2        # long-term mean of variance under risk-neutral dynamics
v0 = 0.25**2           # initial variance under risk-neutral dynamics
rho = 0.7              # correlation between returns and variances under risk-neutral dynamics
sigma = 0.6            # volatility of volatility


theta, v0

In [ ]:
"""
This code simulates the Heston model using a discrete-step numerical approach
to generate price trajectories of a financial asset via:

Correlated Brownian motions:
Simulated with a covariance matrix specified by ρ.
Euler-Maruyama method:
Used to approximate the stochastic differential equations
of the Heston model.
A matrix of correlated random numbers is generated using a
multivariate normal distribution.
the variance evolves stochastically with a non-negativity constraint (using
np.maximun)
"""

def heston_model_sim(S0, v0, rho, kappa, theta, sigma,T, N, M):
    """
    Inputs:
     - S0, v0: initial parameters for asset and variance
     - rho   : correlation between asset returns and variance
     - kappa : rate of mean reversion in variance process
     - theta : long-term mean of variance process
     - sigma : vol of vol / volatility of variance process
     - T     : time of simulation
     - N     : number of time steps
     - M     : number of scenarios / simulations

    Outputs:
    - asset prices over time (numpy array)
    - variance over time (numpy array)
    """
    # initialise other parameters
    dt = T/N
    mu = np.array([0,0])
    cov = np.array([[1,rho],
                    [rho,1]])

    # arrays for storing prices and variances
    S = np.full(shape=(N+1,M), fill_value=S0)
    v = np.full(shape=(N+1,M), fill_value=v0)

    # sampling correlated brownian motions under risk-neutral measure
    Z = np.random.multivariate_normal(mu, cov, (N,M))

    for i in range(1,N+1):
        S[i] = S[i-1] * np.exp( (r - 0.5*v[i-1])*dt + np.sqrt(v[i-1] * dt) * Z[i-1,:,0] )
        v[i] = np.maximum(v[i-1] + kappa*(theta-v[i-1])*dt + sigma*np.sqrt(v[i-1]*dt)*Z[i-1,:,1],0)

    return S, v

In [ ]:
"""
Comparing the impact of opposite correlations
between the Brownian price and variance motions in a Heston simulation.
"""

rho_p = 0.97   # positive correlation value
rho_n = -0.97  # negative correlation value
S_p,v_p = heston_model_sim(S0, v0, rho_p, kappa, theta, sigma,T, N, M)
S_n,v_n = heston_model_sim(S0, v0, rho_n, kappa, theta, sigma,T, N, M)

# implementing mirroring of volatility
v_mirrored = -v_p
v_mirrored = np.maximum(v_mirrored, 0)  # should be positive

S_mirrored, _ = heston_model_sim(S0, v_mirrored[0,0], rho, kappa, theta, sigma, T, N, M)

In [ ]:
# chart for positive correlation value
fig, (ax1, ax2)  = plt.subplots(1, 2, figsize=(12,5))
time = np.linspace(0,T,N+1)
ax1.plot(time,S_p, lw=0.5) # remove lw=0.5 for normal size lines
ax1.grid(True)
ax1.set_title('Heston Model Asset Prices (p) [1.000 simul]')
ax1.set_xlabel('Time')
ax1.set_ylabel('Asset Prices')

ax2.plot(time,v_p, lw=0.5) # remove lw=0.5 for normal size lines
ax2.set_title('Heston Model Variance Process (p) [1.000 simul]')
ax2.set_xlabel('Time')
ax2.set_ylabel('Variance')
ax2.grid(True)
plt.show()

# chart for negative correlation value
fig, (ax1, ax2)  = plt.subplots(1, 2, figsize=(12,5))
time = np.linspace(0,T,N+1)
ax1.plot(time,S_n, lw=0.5) # remove lw=0.5 for normal size lines
ax1.grid(True)
ax1.set_title('Heston Model Asset Prices (n) [1.000 simul]')
ax1.set_xlabel('Time')
ax1.set_ylabel('Asset Prices')

ax2.plot(time,v_n, lw=0.5) # remove lw=0.5 for normal size lines
ax2.set_title('Heston Model Variance Process (n) [1.000 simul]')
ax2.set_xlabel('Time')
ax2.set_ylabel('Variance')
ax2.grid(True)
plt.show()

In [ ]:
# Max number of traces allowed
# Chart for positive correlation value
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
time = np.linspace(0, T, N + 1)

# Lock for max number of traces
max_traces = 1
ax1.plot(time, S_p[:, :max_traces])
ax1.grid(True)
ax1.set_title('Heston Model Asset Prices (p) [1 random simul]')
ax1.set_xlabel('Time')
ax1.set_ylabel('Asset Prices')

ax2.plot(time, v_p[:, :max_traces])
ax2.set_title('Heston Model Variance Process (p) [1 random simul]')
ax2.set_xlabel('Time')
ax2.set_ylabel('Variance')
ax2.grid(True)

plt.tight_layout()
plt.show()

# Chart for negative correlation value
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Lock for max number of traces
ax1.plot(time, S_n[:, :max_traces])
ax1.grid(True)
ax1.set_title('Heston Model Asset Prices (n) [1 random simul]')
ax1.set_xlabel('Time')
ax1.set_ylabel('Asset Prices')

ax2.plot(time, v_n[:, :max_traces])
ax2.grid(True)
ax2.set_title('Heston Model Variance Process (n) [1 random simul]')
ax2.set_xlabel('Time')
ax2.set_ylabel('Variance')

plt.tight_layout()
plt.show()

In [ ]:
# plotting the mirroring
S_mirrored, v_mirrored_new = heston_model_sim(S0, v_mirrored[0, 0], rho, kappa, theta, sigma, T, N, M)

# comparison of charts
fig, ax = plt.subplots()
time = np.linspace(0, T, N+1)

ax.plot(time, S_p[:, 0], label='Original Trajectory')
ax.plot(time, S_mirrored[:, 0], label='Mirrored Trajectory')
ax.set_title('Comparison of Original and Mirrored Trajectories')
ax.set_xlabel('Time')
ax.set_ylabel('Asset Price')
ax.legend()
plt.grid(True)
plt.show()

In [ ]:
"""
Compare the distribution of final prices obtained from the Heston model
with opposite correlations
compared to the distribution of final prices obtained
from a Geometric Brownian Motion (GBM) process.
"""
gbm = S0*np.exp( (r - theta**2/2)*T + np.sqrt(theta)*np.sqrt(T)*np.random.normal(0,1,M) )

fig, ax = plt.subplots()

ax = sns.kdeplot(S_p[-1], label=r"$\rho= 0.97$", ax=ax)
ax = sns.kdeplot(S_n[-1], label=r"$\rho= -0.97$", ax=ax)
ax = sns.kdeplot(gbm, label="GBM", ax=ax)

plt.title(r'Asset Price Density under Heston Model')
plt.xlim([20, 180])
plt.xlabel('$S_T$')
plt.ylabel('Density')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
rho = -0.7
S,v = heston_model_sim(S0, v0, rho, kappa, theta, sigma,T, N, M)

# Set strikes and complete MC option price for different strikes
K = np.arange(20,180,2)

puts = np.array([np.exp(-r*T)*np.mean(np.maximum(k-S,0)) for k in K])
calls = np.array([np.exp(-r*T)*np.mean(np.maximum(S-k,0)) for k in K])

put_ivs = implied_vol(puts, S0, K, T, r, flag='p', q=0, return_as='numpy', on_error='ignore')
call_ivs = implied_vol(calls, S0, K, T, r, flag='c', q=0, return_as='numpy')

plt.plot(K, call_ivs, label=r'IV calls')
plt.plot(K, put_ivs, label=r'IV puts')

plt.ylabel('Implied Volatility')
plt.xlabel('Strike')

plt.title('Implied Volatility Smile from Heston Model')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# the same code is refactored for performance purposed.
# refactoring time: 02 02 2025

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from py_vollib_vectorized import vectorized_implied_volatility as implied_vol

# Model Parameters

# Market parameters
S0 = 100.0    # initial asset price
T  = 1.0      # time to maturity in years
r  = 0.05     # risk-free rate

# Simulation parameters
N = 252       # number of time steps
M = 1000      # number of simulation paths

# Heston model parameters
kappa = 2.7             # rate of mean reversion for the variance process
theta = (0.20)**2       # long-term variance level
v0    = (0.25)**2       # initial variance
sigma = 0.6             # volatility of volatility



# Heston Model Simulation Using Log-Price Transformation

def simulate_heston_log(S0, v0, r, kappa, theta, sigma, rho, T, N, M):
    """
    Simulates the Heston model using the Euler-Maruyama method on the log asset price.

    Parameters:
      S0    : initial asset price
      v0    : initial variance
      r     : risk-free rate
      kappa : mean-reversion speed of the variance process
      theta : long-run mean variance
      sigma : volatility of volatility
      rho   : correlation between asset price and variance Brownian motions
      T     : total time (in years)
      N     : number of time steps
      M     : number of simulation paths

    Returns:
      S       : simulated asset prices (array of shape (N+1, M))
      variance: simulated variance process (array of shape (N+1, M))
    """
    dt = T / N  # time increment

    # Pre-allocate arrays for log asset prices and variance
    log_S   = np.zeros((N+1, M))
    log_S[0] = np.log(S0)
    variance = np.zeros((N+1, M))
    variance[0] = v0

    # Generate independent normal increments scaled by sqrt(dt)
    dW1 = np.random.normal(0, np.sqrt(dt), size=(N, M))
    dZ  = np.random.normal(0, np.sqrt(dt), size=(N, M))
    # Construct the second Brownian increment to have correlation rho with dW1
    dW2 = rho * dW1 + np.sqrt(1 - rho**2) * dZ

    # Time stepping loop using Euler-Maruyama discretization
    for i in range(1, N+1):
        # Update variance; ensure non-negativity by applying np.maximum
        variance[i] = variance[i-1] + kappa * (theta - variance[i-1]) * dt \
                      + sigma * np.sqrt(np.maximum(variance[i-1], 0)) * dW2[i-1]
        variance[i] = np.maximum(variance[i], 0)

        # Update log asset price using the previous variance value
        log_S[i] = log_S[i-1] + (r - 0.5 * variance[i-1]) * dt \
                   + np.sqrt(np.maximum(variance[i-1], 0)) * dW1[i-1]

    # Convert log asset prices back to actual asset prices
    S = np.exp(log_S)
    return S, variance



# Simulations for Different Correlation Scenarios

# Simulate for a high positive correlation
rho_pos = 0.97
S_pos, var_pos = simulate_heston_log(S0, v0, r, kappa, theta, sigma, rho_pos, T, N, M)

# Simulate for a high negative correlation
rho_neg = -0.97
S_neg, var_neg = simulate_heston_log(S0, v0, r, kappa, theta, sigma, rho_neg, T, N, M)


# Mirrored Volatility Scenario

# Here we "mirror" the variance from the positive correlation simulation
# (i.e. multiply by -1 and then force non-negativity) and then simulate asset
# prices using this altered initial variance.
mirrored_variance = np.maximum(-var_pos, 0)
# Use the mirrored initial variance value (typically zero if v0 > 0) to simulate a new path.
S_mirrored, _ = simulate_heston_log(S0, mirrored_variance[0, 0], r, kappa, theta, sigma, rho_pos, T, N, M)

# Plotting the Simulation Results

time_grid = np.linspace(0, T, N+1)

# (1) Plot trajectories for the positive correlation simulation
fig1, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ax1.plot(time_grid, S_pos, lw=0.5)
ax1.set_title('Asset Price Trajectories (ρ = 0.97)')
ax1.set_xlabel('Time (years)')
ax1.set_ylabel('Asset Price')
ax1.grid(True)

ax2.plot(time_grid, var_pos, lw=0.5)
ax2.set_title('Variance Process (ρ = 0.97)')
ax2.set_xlabel('Time (years)')
ax2.set_ylabel('Variance')
ax2.grid(True)
plt.tight_layout()
plt.show()

# (2) Plot trajectories for the negative correlation simulation
fig2, (ax3, ax4) = plt.subplots(1, 2, figsize=(12, 5))
ax3.plot(time_grid, S_neg, lw=0.5)
ax3.set_title('Asset Price Trajectories (ρ = -0.97)')
ax3.set_xlabel('Time (years)')
ax3.set_ylabel('Asset Price')
ax3.grid(True)

ax4.plot(time_grid, var_neg, lw=0.5)
ax4.set_title('Variance Process (ρ = -0.97)')
ax4.set_xlabel('Time (years)')
ax4.set_ylabel('Variance')
ax4.grid(True)
plt.tight_layout()
plt.show()

# (3) Compare a single asset price path: original vs. mirrored volatility
fig3, ax5 = plt.subplots(figsize=(10, 6))
ax5.plot(time_grid, S_pos[:, 0], label='Original Trajectory', lw=2)
ax5.plot(time_grid, S_mirrored[:, 0], label='Mirrored Trajectory', lw=2, linestyle='--')
ax5.set_title('Comparison of Original and Mirrored Asset Price Trajectories')
ax5.set_xlabel('Time (years)')
ax5.set_ylabel('Asset Price')
ax5.legend()
ax5.grid(True)
plt.show()

# (4) Compare the distribution of final asset prices with a GBM process
# Generate GBM final asset prices for comparison
gbm_final = S0 * np.exp((r - 0.5 * theta) * T + np.sqrt(theta * T) * np.random.normal(0, 1, M))
final_prices_pos = S_pos[-1, :]

fig4, ax6 = plt.subplots(figsize=(10, 6))
sns.kdeplot(final_prices_pos, label=r"Heston (ρ = 0.97)", ax=ax6)
sns.kdeplot(S_neg[-1, :], label=r"Heston (ρ = -0.97)", ax=ax6)
sns.kdeplot(gbm_final, label="GBM", ax=ax6)
ax6.set_title('Distribution of Final Asset Prices')
ax6.set_xlabel('Final Asset Price ($S_T$)')
ax6.set_ylabel('Density')
ax6.grid(True)
ax6.legend()
plt.xlim(20, 180)
plt.show()


# Option Pricing and Implied Volatility Smile

# For this section, we use the negative correlation simulation's final asset prices.
S_final_neg = S_neg[-1, :]

# Define a range of strikes for option pricing
strikes = np.arange(20, 180, 2)

# Compute Monte Carlo prices for European calls and puts (discounted)
call_prices = np.array([np.exp(-r * T) * np.mean(np.maximum(S_final_neg - k, 0)) for k in strikes])
put_prices  = np.array([np.exp(-r * T) * np.mean(np.maximum(k - S_final_neg, 0)) for k in strikes])

# Calculate the implied volatilities using py_vollib_vectorized
call_iv = implied_vol(call_prices, S0, strikes, T, r, flag='c', q=0, return_as='numpy')
put_iv  = implied_vol(put_prices, S0, strikes, T, r, flag='p', q=0, return_as='numpy', on_error='ignore')

# Plot the implied volatility smile
fig5, ax7 = plt.subplots(figsize=(10, 6))
ax7.plot(strikes, call_iv, label='Call Implied Volatility', marker='o')
ax7.plot(strikes, put_iv, label='Put Implied Volatility', marker='o')
ax7.set_title('Implied Volatility Smile from Heston Model')
ax7.set_xlabel('Strike Price')
ax7.set_ylabel('Implied Volatility')
ax7.grid(True)
ax7.legend()
plt.show()

# Jump Diffusion Model
## Chapter V

In [ ]:
"""
Notation for variables:
S = stock price
K = strike price
T = time to maturity (y)
σ = vol
m = jump mean size
v = st dev of jump size
λ = intensity of Poisson process (jumps per year)
dW(t) = Wiener process
N(t) = compound Poisson process
Vbs = value of option with B&S
Vmjd = value of option with Merton Jump Diffusion model

"""
import matplotlib.pyplot as plt
import numpy as np

# troubleshooting in case matplot crashes
# import importlib
# importlib.reload(plt)


S = 100           # current stock price
T = 1             # time to maturity
r = 0.02          # risk free rate
m = 0             # mean of jump size
v = 0.3           # standard deviation of jump
lam =3            # intensity
steps =10000      # time steps
Npaths = 1        # number of paths to simulate
sigma = 0.2       # annual standard deviation


In [ ]:
# define a Merton Jump function
def merton_jump_paths(S, T, r, sigma,  lam, m, v, steps, Npaths):
    size=(steps,Npaths)
    dt = T/steps
    poi_rv = np.multiply(np.random.poisson( lam*dt, size=size),
                         np.random.normal(m,v, size=size)).cumsum(axis=0)
    geo = np.cumsum(((r -  sigma**2/2 -lam*(m  + v**2*0.5))*dt +\
                              sigma*np.sqrt(dt) * \
                              np.random.normal(size=size)), axis=0)

    return np.exp(geo+poi_rv)*S

In [ ]:
j = merton_jump_paths(S, T, r, sigma, lam, m, v, steps, Npaths)

plt.xlabel('Days')
plt.ylabel('Stock Price')
plt.grid(True)
plt.title('Jump Diffusion Process')
plt.plot(j)
plt.show()

In [ ]:
# the same code is refactored for performance purposed.
# refactoring time: 02 02 2025

import numpy as np
import matplotlib.pyplot as plt

# =============================================================================
# Simulation Parameters
# =============================================================================
S0 = 100.0      # Initial stock price
T = 1.0         # Time to maturity in years
r = 0.02        # Risk-free interest rate
sigma = 0.2     # Annual volatility of the continuous diffusion component
lam = 5         # Intensity (jumps per year)
m = 0           # Mean of the jump size (in logarithmic terms)
v = 0.3         # Standard deviation of the jump size
steps = 10000   # Number of time steps in the simulation
Npaths = 1      # Number of simulation paths

# =============================================================================
# Function: simulate_jump_diffusion
# =============================================================================
def simulate_jump_diffusion(S0, T, r, sigma, lam, m, v, steps, Npaths):
    """
    Simulates a jump diffusion process using the Merton jump diffusion model.

    The stock price process is modeled as:

        S(t) = S0 * exp{ (r - 0.5*sigma^2 - λ*(E[J]-1)) * t + sigma*W(t) + Σ_{i=1}^{N(t)} Y_i }

    where:
      - W(t) is a Wiener process,
      - N(t) is a Poisson process with intensity λ,
      - Y_i ~ N(m, v^2) represents the logarithmic jump sizes, and
      - E[J] = exp(m + 0.5*v^2) is the expected jump multiplier.

    The term λ*(E[J]-1) adjusts the drift to compensate for the average jump effect.

    Parameters:
      S0     : Initial stock price
      T      : Time to maturity (years)
      r      : Risk-free rate
      sigma  : Volatility of the continuous diffusion component
      lam    : Intensity of the jump process (jumps per year)
      m      : Mean of the jump size (in log terms)
      v      : Standard deviation of the jump size
      steps  : Number of time steps in the simulation
      Npaths : Number of simulation paths

    Returns:
      S_paths: Simulated stock price paths as a numpy array of shape (steps+1, Npaths)
    """
    dt = T / steps  # Time increment per step

    # --- Continuous Diffusion Component ---
    # Generate increments of the Wiener process (Brownian motion)
    dW = np.random.normal(0, np.sqrt(dt), size=(steps, Npaths))

    # Compute the drift adjustment for the continuous part.
    # The compensation term is: λ*(E[J]-1) = λ*(exp(m + 0.5*v^2) - 1)
    drift_compensation = lam * (np.exp(m + 0.5 * v**2) - 1)
    continuous_drift = (r - 0.5 * sigma**2 - drift_compensation) * dt
    continuous_component = continuous_drift + sigma * dW

    # --- Jump Component ---
    # For each time step, simulate the number of jumps (Poisson-distributed)
    jump_counts = np.random.poisson(lam * dt, size=(steps, Npaths))

    # For simplicity, approximate the total jump effect in each time interval
    # by multiplying one normal random draw by the number of jumps.
    # (In very small time intervals, the probability of multiple jumps is low.)
    jump_component = jump_counts * np.random.normal(m, v, size=(steps, Npaths))

    # --- Combine Both Components ---
    # The total log-return is the sum of the continuous and jump components.
    log_returns = continuous_component + jump_component

    # Compute the cumulative sum to get the log-price path.
    # Start at 0 (i.e. log(S0) = log(S0)).
    log_S = np.vstack([np.zeros((1, Npaths)), np.cumsum(log_returns, axis=0)])

    # Convert log prices back to actual stock prices.
    S_paths = S0 * np.exp(log_S)
    return S_paths

# =============================================================================
# Generate and Plot the Jump Diffusion Process
# =============================================================================
jump_paths = simulate_jump_diffusion(S0, T, r, sigma, lam, m, v, steps, Npaths)

plt.figure(figsize=(10, 6))
plt.plot(jump_paths, lw=1.5)
plt.xlabel('Time Steps')
plt.ylabel('Stock Price')
plt.title('Jump Diffusion Process (Merton Model)')
plt.grid(True)
plt.show()

# Ornstein-Uhlenbeck
## Annexes


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parametri del processo
theta_values = [0.1, 1.5, 5]  # Diversi valori di mean reversion
mu = 0.5  # Valore medio a cui il processo torna
sigma = 0.2  # Volatilità
X0 = 0.0  # Valore iniziale
T = 10  # Orizzonte temporale
dt = 0.01  # Passo temporale
N = int(T / dt)  # Numero di passi
t = np.linspace(0, T, N)  # Asse temporale

# Simulazione del processo per diversi theta
plt.figure(figsize=(10, 6))
for theta in theta_values:
    X = np.zeros(N)
    X[0] = X0
    for i in range(1, N):
        dW = np.sqrt(dt) * np.random.randn()  # Incremento di Wiener
        X[i] = X[i - 1] + theta * (mu - X[i - 1]) * dt + sigma * dW

    plt.plot(t, X, label=f'θ = {theta}')

# Grafico
plt.xlabel('Time')
plt.ylabel('X_t')
plt.title('Ornstein-Uhlenbeck process for different mean reversion values')
plt.axhline(y=mu, color='k', linestyle='--', label='Mean level μ')
plt.legend()
plt.grid()
plt.show()

In [ ]:
"""
Subsequent run of Brownian motion
for graphic purposes.
The request is to be able to show a limited number of paths.
"""

# Brownian motion with Euler

# Simulation with euler-maruyama method

import numpy as np
import matplotlib.pyplot as plt

# Parameters
num_paths = 5000                  # Number of Brownian motion paths
num_steps = 1000                  # Number of time steps
T = 1.0                           # Time horizon
dt = T / num_steps                # Time step size
t = np.linspace(0, T, num_steps)  # Time intervals

# Euler-Maruyama method for Brownian motion
np.random.seed(66)  # For reproducibility
dW = np.sqrt(dt) * np.random.randn(num_paths, num_steps)  # Increments
W = np.cumsum(dW, axis=1)  # Cumulative sum to build W(t)

# Add initial value W(0) = 0
W = np.hstack((np.zeros((num_paths, 1)), W))

# Plot a sample of Brownian motion paths
plt.figure(figsize=(10, 6))
for i in range(min(3, num_paths)):  # Show only 30 trajectories
    plt.plot(t, W[i, :-1])
plt.title("Brownian Motion Simulations (Euler Method) [3 paths]")
plt.xlabel("Time")
plt.ylabel("$W(t)$")
plt.grid(True)
plt.show()



# Backup - Extra


# EUR/CHF Crash



In [ ]:


import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

# Definisci l'intervallo di date
start_date = '2014-09-12'
end_date = '2015-08-19'

# Scarica i dati storici per EUR/CHF
ticker = 'EURCHF=X'
data = yf.download(ticker, start=start_date, end=end_date)

# Verifica se i dati sono stati scaricati correttamente
if data.empty:
    print("Nessun dato trovato per l'intervallo di date specificato.")
else:
    # Crea il grafico dei prezzi di chiusura
    plt.figure(figsize=(12, 6))
    plt.plot(data.index, data['Close'], label='Closing Price')
    plt.title('EUR/CHF FX Daily Rate')
    plt.xlabel('Date')
    plt.ylabel('Closing price')
    plt.legend()
    plt.grid(True)
    plt.show()